# Task 3: antiboy protein classifcation

Goal: assign each 4 letter mutation sequence to a 0 or 1 activation

example:

    DKWL, 0
    LCLA, 1
    
    
The same letter can appear multiple times
The ordering matters
n = 21 (amount of amino acids)

    - How do we abstract the letters that describe the sequence? 2 options:
        - every letter gets assigned a number, each sequence is a 1D vector with 4 numbers
        - or every data point is 21 dimeonsonal vector amount of each amino acid present in mutation (no order)
        
Classes are very imbalanced (few positive cases), therefore we need to up- or downsample, as class weights are not implemented in MLP

In [141]:
import numpy as np
import time
import random
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import normalize

seq_conv = lambda seq: np.asarray(list(map(amino_conv,list(seq))))

### Get data 
#### turns data into an ndarray as containing the sequence as np array of amino acids acording to their number from 1 - 21 and then normalized (DKWL -> [0.4,0.14,0.5,0.1]) (this might have to be changed to a standardized way)


In [142]:
def amino_conv(a):
    if(a == 65):
        return 1
    elif(a==86):
        return 2
    elif(a==73):
        return 3
    elif(a==76):
        return 4
    elif(a==77):
        return 5
    elif(a==70):
        return 6
    elif(a==89):
        return 7
    elif(a==87):
        return 8
    elif(a==82):
        return 9
    elif(a==72):
        return 10
    elif(a==75):
        return 11
    elif(a==68):
        return 12
    elif(a==69):
        return 13
    elif(a==83):
        return 14
    elif(a==84):
        return 15
    elif(a==78):
        return 16
    elif(a==81):
        return 17
    elif(a==67):
        return 18
    elif(a==85):
        return 19
    elif(a==71):
        return 20
    elif(a==80):
        return 21
    else:
        print ("--------------------------\nERROR IN DATA, UNDEFINED AMINO ACID for %d!\n--------------------------" %a)
        return -1
        

In [143]:
def balance_data(sequences,activations,dup):
    n = activations.size
    k = np.sum(activations)
    ones_act = np.zeros(k)
    ones_sequ = np.zeros((k,4))
    count = 0
    for i in range(n):
        if (activations[i] == 1):
            ones_act[count] = activations[i]
            ones_sequ[count] = sequences[i]
            count = count + 1
    res_act = np.zeros(n + k * dup)
    res_sequ = np.zeros((n + k * dup,4))
    res_act[0:n] = activations
    res_sequ[0:n] = sequences
    for i in range(dup):
        res_act[n + i * k: n + (i+1) *k] = ones_act
        res_sequ[n + i * k: n + (i+1) *k] = ones_sequ
    return res_sequ, res_act

In [148]:
def get_predict_data():
    
    data = np.genfromtxt("./data/test.csv",delimiter=",",skip_header=1,
                          dtype=[('seq','object')],converters={0: seq_conv})
    return data

In [149]:
def get_data(dup):
    start_time = time.time()
    
    
    data = np.genfromtxt("./data/train.csv",delimiter=",",skip_header=1,
                         dtype=[('seq','object'),('active','int')],converters={0: seq_conv})
    m = data.size
    sequences = np.zeros((m,4))
    for i in range(m):
        sequences[i] = data[i][0]
        
    elapsed_time = time.time() - start_time
    sequences, activations = balance_data(sequences,data['active'],dup)
    print ("-----------------------------------------------\n Setup time is: %d seconds\n-----------------------------------------------" % elapsed_time)
    return sequences, activations
    

In [150]:
#return percentage of positive actvations(class 1)
def ratio(activations):
    return np.sum(activations) / activations.size


### For training model first intuition points to trainin neural network
    -Multi layer perceptron
    
#### For validation we use gridearch crossvalidation to tune hyperparameters


In [151]:
def mlp_model():
    return MLPClassifier(solver='sgd',learning_rate='adaptive',alpha=1e-5, activation='relu')

In [219]:
def cross_model():
    return MLPClassifier(solver='sgd',learning_rate='adaptive',activation='relu',hidden_layer_sizes=(50,20), alpha=0.001)

In [220]:
def grid_model():
        
    parameters =  {
                    'hidden_layer_sizes':[(200,100,50),(100,50,20,10,10)]
                  } 
    GridSearchCV(mlp_model(),parameters,scoring='f1', cv=3)
    return 

In [221]:
def cross_val(sequences, activations):
    start_time = time.time()
      
    res = cross_val_score(cross_model(), sequences,activations,cv=3,scoring='f1')
    
    elapsed_time = time.time() - start_time
    print ("-----------------------------------------------\n Training time is: %s seconds\n-----------------------------------------------" % elapsed_time)
    return res

In [222]:
def grid_val(sequences, activations):
    start_time = time.time()
      
    res = cross_model().fit(sequences,activations)
    
    elapsed_time = time.time() - start_time
    print ("-----------------------------------------------\n Training time is: %s seconds\n-----------------------------------------------" % elapsed_time)
    return res


In [223]:
def train(model,sequences,activations):
    return model.fit(sequences,activations)

In [224]:
def make_predictions(model, sequences):
    res = model.predict(sequences)
    np.savetxt("prediction.csv",res, delimiter=",",fmt='%i',comments='')

### Driver code

In [225]:
def main():
    sequences,activations = get_data(24)#duplicates ones 24 times such that classes are balanced
    pred_sequences = get_predict_data()
    print(ratio(activations))
    clf = train(cross_model(),sequences,activations)
#     make_predictions(clf,pred_sequences)
    print(cross_val(sequences,activations))


In [226]:
main()

-----------------------------------------------
 Setup time is: 1 seconds
-----------------------------------------------
0.49422369458313


C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


-----------------------------------------------
 Training time is: 271.7117772102356 seconds
-----------------------------------------------
[0.89445457 0.89010243 0.88034405]


C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
